# Groover's serach for RNA design

In [1]:
import numpy as np
from math import pi,log2,floor,ceil
from qiskit import *
from qiskit.circuit import *
from qiskit.extensions import *
from qiskit.circuit.library import *
from qiskit.extensions.simulator.snapshot import snapshot
from qiskit.quantum_info.operators import Operator
from qiskit.extensions.simulator.snapshot import snapshot
from scipy import optimize
from matplotlib.pyplot import plot,show
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

## 1 - Addition

Let's implement the circuit in [1] modified to add mod 2^n.

In [2]:
#MAJ
q=QuantumRegister(3)
MAJ = QuantumCircuit(q)

MAJ.cnot(q[2],q[1])
MAJ.cnot(q[2],q[0])
MAJ.ccx(q[0],q[1],q[2])

maj = MAJ.to_gate(label='MAJ')

#UMA
q=QuantumRegister(3)
UMA = QuantumCircuit(q)

UMA.ccx(q[0],q[1],q[2])
UMA.cnot(q[2],q[0])
UMA.cnot(q[0],q[1])

uma = UMA.to_gate(label='UMA')

#add
def add_circuit(nb_bits):
    concerved = QuantumRegister(nb_bits, name="=")
    not_concerved = QuantumRegister(nb_bits, name="->")
    initial_carry = QuantumRegister(1)
    add = QuantumCircuit(initial_carry,concerved,not_concerved)
    add.append(maj,[initial_carry,not_concerved[0],concerved[0]])
    for i in range(1,nb_bits):
        add.append(maj,[concerved[i-1],not_concerved[i],concerved[i]])
    for i in range(nb_bits-1,0,-1):
        add.append(uma,[concerved[i-1],not_concerved[i],concerved[i]])
    add.append(uma,[initial_carry,not_concerved[0],concerved[0]])
    return add

def add_gate(nb_bits):
    return add_circuit(nb_bits).to_gate(label="add{}".format(nb_bits))

add_circuit(6).draw()

┌──────┐                                                                »
  q2: ┤0     ├────────────────────────────────────────────────────────────────»
      │      │┌──────┐                                                        »
 =_0: ┤2     ├┤0     ├────────────────────────────────────────────────────────»
      │      ││      │┌──────┐                                                »
 =_1: ┤      ├┤2     ├┤0     ├────────────────────────────────────────────────»
      │      ││      ││      │┌──────┐                                ┌──────┐»
 =_2: ┤      ├┤      ├┤2     ├┤0     ├────────────────────────────────┤0     ├»
      │  MAJ ││      ││      ││      │┌──────┐                ┌──────┐│      │»
 =_3: ┤      ├┤      ├┤      ├┤2     ├┤0     ├────────────────┤0     ├┤2     ├»
      │      ││  MAJ ││      ││      ││      │┌──────┐┌──────┐│      ││      │»
 =_4: ┤      ├┤      ├┤      ├┤      ├┤2     ├┤0     ├┤0     ├┤2     ├┤      ├»
      │      ││      ││  MAJ ││      ││      ││      ││      ││      ││      │»
 =_5: ┤      ├┤      ├┤      ├┤      ├┤      ├┤2     ├┤2     ├┤      ├┤      ├»
      │      ││      ││      ││  MAJ ││      ││      ││      ││      ││  UMA │»
->_0: ┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
      └──────┘│      ││      ││      ││  MAJ ││      ││      ││  UMA ││      │»
->_1: ────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
              └──────┘│      ││      ││      ││  MAJ ││  UMA ││      ││      │»
->_2: ────────────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
                      └──────┘│      ││      ││      ││      ││      ││      │»
->_3: ────────────────────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤1     ├»
                              └──────┘│      ││      ││      ││      │└──────┘»
->_4: ────────────────────────────────┤1     ├┤      ├┤      ├┤1     ├────────»
                                      └──────┘│      ││      │└──────┘        »
->_5: ────────────────────────────────────────┤1     ├┤1     ├────────────────»
                                              └──────┘└──────┘                »
«                      ┌──────┐
«  q2: ────────────────┤0     ├
«              ┌──────┐│      │
« =_0: ────────┤0     ├┤2     ├
«      ┌──────┐│      ││      │
« =_1: ┤0     ├┤2     ├┤      ├
«      │      ││      ││      │
« =_2: ┤2     ├┤      ├┤      ├
«      │      ││      ││  UMA │
« =_3: ┤      ├┤      ├┤      ├
«      │      ││  UMA ││      │
« =_4: ┤      ├┤      ├┤      ├
«      │  UMA ││      ││      │
« =_5: ┤      ├┤      ├┤      ├
«      │      ││      ││      │
«->_0: ┤      ├┤      ├┤1     ├
«      │      ││      │└──────┘
«->_1: ┤      ├┤1     ├────────
«      │      │└──────┘        
«->_2: ┤1     ├────────────────
«      └──────┘                
«->_3: ────────────────────────
«                              
«->_4: ────────────────────────
«                              
«->_5: ────────────────────────
«

## 2 - Matchnig parentesis

`(`, `.` and `)` are encoded respectively by 1, 0 and -1, which are encoded with tow's complement representation.

In [6]:
nb_bits_bracket_dot = 2
nb_bits_loop_type = 3

def n_to_ctrl_state(n,nb_bits):
    return format(n,"0{}b".format(nb_bits))

#if test_concerved == tested_to_0 then tested_to_0 is set to 0 and will trigger the application of some gate in then_append(circuit,tested_to_0,nb_bits_test,gate,...)
def start_if_equal(circuit,tested_to_0,nb_bits_test,test_concerved):
    for i in range(nb_bits_test):
        circuit.cx(test_concerved[i],tested_to_0[i])

end_if_equal=start_if_equal

def then_append(circuit,test_0,nb_bits_test,gate,qbits_list):
    circuit.append(gate.control(nb_bits_test,ctrl_state="0"*nb_bits_test),[test_0[i] for i in range(nb_bits_test)]+qbits_list)
    return    
    
def matching_parenthesis_circuit(length,truncation_point=-1):
    nb_bits_position=ceil(log2(length+1)) #length+1 because we add a ( at the begining to check the folding is well formed
    add_positions=add_gate(nb_bits_position)
    nb_bits_extend = max(0,nb_bits_position-nb_bits_bracket_dot)
    
    #input
    #semantics of folding[0:nb_bits_bracket_dot]
    #. == "00"
    #h == "10"
    #) == "11"
    #( == "01"
    folding = QuantumRegister(nb_bits_bracket_dot+nb_bits_extend,name="(/./h/)")
    
    #auxillary qbits
    k_folding = QuantumRegister(nb_bits_position,name="k_folding")
    count = QuantumRegister(nb_bits_position,name="count") #count how many parenthesis still need to be closed. folding[k_folding=0] is initiallised to ( and count[k_count=0] is used to check that there is no extra ) that would match in the folding
    k_count = QuantumRegister(nb_bits_position,name="k_count")
    continue_searching_right_matching_left = QuantumRegister(1,name="continue_match") #continue_searching_right_matching_left[k] corresponds to count[k]
    
    change_folding_encoding = QuantumRegister(1,name="change_encoding")
    continue_scanning = QuantumRegister(1,name="continue_scan")
    zeros = QuantumRegister(nb_bits_position,name="zero")
    one = QuantumRegister(1,name="one")
    
    #results
    k_right_matching_left = QuantumRegister(nb_bits_position,name="k_match") #k_right_matching_left[k_left] is initialised to k_left and is incremented until count[k_left]==0, that is until k_right_matching_left[k_left] is the indice of the ) matching the ( == folding[k_left]
    loop = QuantumRegister(nb_bits_position,name="loop") #length of the loop starting at folding[k_left]==(. loop[k_left] is initialised to 0 and is incremented each time count[k_left]==1 until count[k_left]==0
    #semantics of loop_type
    #interior -> ?
    #buggle   -> ?
    #multi    -> ?
    #hairpin  -> ?
    #stack    -> ?
    loop_type = QuantumRegister(nb_bits_loop_type,name="loop_type")
    incorrect_folding = QuantumRegister(nb_bits_position,name="incorrect")
    correct_folding = QuantumRegister(1,name="correct")
    
    ####circuit = QuantumCircuit(change_folding_encoding,folding,k_folding,count,k_count,continue_scanning,continue_searching_right_matching_left,one,zeros,k_right_matching_left,loop,loop_type,incorrect_folding,correct_folding) #previous order of registers
    #circuit = QuantumCircuit(correct_folding,incorrect_folding,loop,continue_scanning,continue_searching_right_matching_left,zeros,one,k_right_matching_left,count,k_count,change_folding_encoding,k_folding,folding) #order of registers as they appear along the program
    circuit = QuantumCircuit()
    
    def semantics(bit_string):
        i_bit = 0
        semantics_string = ""
        
        def semantics_number(label,nb_bits_number):
            s=str(int(bit_string[i_bit:i_bit+nb_bits_number],2))
            delta=nb_bits_number-len(s)
            return s+"_"+label+delta*" "+" ",nb_bits_number
        
        if circuit.has_register(correct_folding):
            s,i=semantics_number("?",1)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(incorrect_folding):
            s,i=semantics_number("!",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(loop_type):
            s,i=semantics_number("lt",nb_bits_loop_type)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(loop):
            s,i=semantics_number("lo",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(continue_scanning):
            s,i=semantics_number("sc",1)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(continue_searching_right_matching_left):
            s,i=semantics_number("se",1)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(zeros):
            semantics_string+="cst"
            semantics_string+=bit_string[i_bit:i_bit+nb_bits_position]
            i_bit+=nb_bits_position
        if circuit.has_register(one):
            semantics_string+=bit_string[i_bit:i_bit+1]
            i_bit+=1
            semantics_string+=" "
        if circuit.has_register(k_right_matching_left):
            s,i=semantics_number("km",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(count):
            s,i=semantics_number("c",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(k_count):
            s,i=semantics_number("kc",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(change_folding_encoding):
            semantics_string+="-" if bit_string[i_bit:i_bit+1]=="0" else "h"
            i_bit+=1
            semantics_string+=" "
        if circuit.has_register(k_folding):
            s,i=semantics_number("kf",nb_bits_position)
            semantics_string+=s
            i_bit+=i
        if circuit.has_register(folding):
            semantics_string+=bit_string[i_bit:i_bit+nb_bits_extend]
            i_bit+=nb_bits_extend
            semantics_string+=("." if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="00" else
                               "h" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="10" else
                               "(" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="01" else
                               ")" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="11" else "")
            i_bit+=nb_bits_bracket_dot
        return semantics_string
    
    correct_folding,incorrect_folding,loop,continue_scanning,continue_searching_right_matching_left,zeros,one,k_right_matching_left,count,k_count,change_folding_encoding,k_folding,folding
    
    
    #1) Superposition at the begining of the Groover search
    
    #any symbol
    circuit.add_register(folding)
    for i in range(nb_bits_bracket_dot):
        circuit.h(folding[i])
    
    if truncation_point == 0 or truncation_point == "any folding":
        return circuit,semantics
    
    #at any position
    circuit.add_register(k_folding)
    circuit.h(k_folding)
    
    if truncation_point == 1 or truncation_point == "any k_folding":
        return circuit,semantics
    
    #except at position "0"*nb_bits_position, where it is necessarily a ( == "01" to check if the folding is well formed
    for i in range(nb_bits_bracket_dot):
        circuit.append(HGate().inverse().control(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[i]])
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[0]])
    
    #except at positions k > length, where folding[k] is set to .
    for k in range(length+1,2**nb_bits_position):
        for i in range(nb_bits_bracket_dot):
            circuit.append(HGate().control(nb_bits_position,ctrl_state=n_to_ctrl_state(k,nb_bits_position)),[k_folding[j] for j in range(nb_bits_position)]+[folding[i]])
    
    if truncation_point == 2 or truncation_point == "candidate folding":
        return circuit,semantics
    
    circuit.add_register(change_folding_encoding)
    
    #if this symbol is h == "10", then change it to be . == "00"
    circuit.ccx(*[folding[i] for i in range(nb_bits_bracket_dot)],change_folding_encoding,ctrl_state="10")
    circuit.cnot(change_folding_encoding,folding[nb_bits_bracket_dot-1])
    
    if truncation_point == 3 or truncation_point == "encoding":
        return circuit,semantics
    
    
    #2) Initialisation of the oracle
    
    #initialise k_count to k_left
    circuit.add_register(k_count)
    circuit.h(k_count)
    
    if truncation_point == 4 or truncation_point == "init k_count":
        return circuit,semantics
    
    #extend folding
    #folding[0:nb_bits_bracket_dot] -> folding[0:nb_bits_position]
    #. == "00" ->  0 == "0...000"
    #do nothing
    #( == "01" ->  1 == "0...001"
    #do nothing
    #) == "11" -> -1 == "1...111"
    if 0 < nb_bits_extend:
        circuit.append(MCMT(XGate(),nb_bits_bracket_dot,nb_bits_extend),[folding[i] for i in range(nb_bits_position)])
    
    if truncation_point == 5 or truncation_point == "extend folding":
        return circuit,semantics
    
    #if k_folding == k_count
    start_if_equal(circuit,k_count,nb_bits_position,k_folding)
    
    #initialise count with folding
    circuit.add_register(count)
    #folding -> count
    #. == "00" -> 0 == "0...000"
    #do nothing
    #) == "11" -> 0 == "0...000"
    #do nothing
    #( == "01" -> 1 == "0...001"
    then_append(circuit,k_count,nb_bits_position,CCXGate(ctrl_state="01"),[folding[0],folding[1],count[0]])
    
    #end if k_folding == k_count
    end_if_equal(circuit,k_count,nb_bits_position,k_folding)
    
    if truncation_point == 6 or truncation_point == "init count":
        return circuit,semantics
    
    #initialise k_right_matching_left[k_count] to k_left (and it will then be incremented up to the matchning k_right)
    circuit.add_register(k_right_matching_left)
    for i in range(nb_bits_position):
        circuit.cx(k_count[i],k_right_matching_left[i])
    
    if truncation_point == 7 or truncation_point == "init k_right_matching_left":
        return circuit,semantics
    
    #initialise one to 1
    circuit.add_register(one)
    circuit.add_register(zeros)
    circuit.x(one)
    
    if truncation_point == 8 or truncation_point == "init cst":
        return circuit,semantics
    
    #initialise continue_searching_right_matching_left=1 when count == (
    #indeed, when folding[k_left] == ( (which will be checked by there <*>) then there is initially 1 parenthesis to be closed
    circuit.add_register(continue_searching_right_matching_left)
    circuit.cnot(count[0],continue_searching_right_matching_left)
    
    if truncation_point == 9 or truncation_point == "init continue_searching_right_matching_left":
        return circuit,semantics
    
    #initialise continue_scanning=1
    circuit.add_register(continue_scanning)
    circuit.x(continue_scanning)
    
    if truncation_point == 10 or truncation_point == "init continue_scanning":
        return circuit,semantics
    
    #circuit.add_register(loop)
    
    #3) Compute parameters by scanning the folding
    
    for k in range(length):
        #<*> if it has been checked that there is a matching ) to find, then continue_searching_right_matching_left[k_left]==1, otherwise continue_searching_right_matching_left[k_left]==0
        
        #increment k_right_matching_left[k_left] if continue_searching_right_matching_left[k_left]==1
        circuit.append(add_positions.control(),[continue_scanning,zeros[nb_bits_position-1],continue_searching_right_matching_left]+[zeros[i] for i in range(nb_bits_position-1)]+[k_right_matching_left[i] for i in range(nb_bits_position)])
        
        #go to the next position
        #increment k_count
        circuit.append(add_positions,[zeros[nb_bits_position-1],one]+[zeros[i] for i in range(nb_bits_position-1)]+[k_count[i] for i in range(nb_bits_position)])
        
        #if the end has been reached, that is if k_count == 0
        #then do not continue_scanning
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_scanning])
        
        #if k_folding == k_count
        start_if_equal(circuit,k_count,nb_bits_position,k_folding)
        
        #if continue_searching_right_matching_left[k_left]==1, then update count[k_left] for the next candidate
        if 0 == k:
            #necessarily if folding[k_left]==( then continue_searching_right_matching_left[k_left]==1
            #so it is ok not controlling by continue_searching_right_matching_left here
            then_append(circuit,k_count,nb_bits_position,add_positions.control()              ,[continue_scanning,zeros[nb_bits_position-1]]+[count[i] for i in range(nb_bits_position)]+[folding[i] for i in range(nb_bits_position)])
        else:
            then_append(circuit,k_count,nb_bits_position,add_positions.control(2)             ,[continue_scanning,continue_searching_right_matching_left,zeros[nb_bits_position-1]]+[count[i] for i in range(nb_bits_position)]+[folding[i] for i in range(nb_bits_position)])
        
        
        #if continue_searching_right_matching_left[k_left]==0, then simply increment count[k_left] so that it never equals 0 again
        #it is ok not controlling by continue_scanning here
        if 0 != k:
            #as stated previously, if 0==k then necessarily if folding[k_left]==( then continue_searching_right_matching_left[k_left]==1
            #so there is no need doing this if 0==k
            then_append(circuit,k_count,nb_bits_position,add_positions.control(ctrl_state="0"),[continue_searching_right_matching_left,zeros[nb_bits_position-1]]+[count[i] for i in range(nb_bits_position)]+[one]+[zeros[i] for i in range(nb_bits_position-1)])
        
        ##if 1 == count[k_left]
        #start_if_equal(circuit,count,nb_bits_position,[one]+[zeros[i] for i in range(nb_bits_position-1)])
        #
        ##if 1 == continue_searching_right_matching_left[k_left] then increment loop[k_left]
        #if 0==k:
        #    #since the value 0==loop[k_left] is known then there is no need to add in order to increment
        #    #a CXGate() is sufficient
        #    then_append(circuit,[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)],2*nb_bits_position,CCXGate(),[continue_scanning,continue_searching_right_matching_left,loop[0]])
        #else:
        #    then_append(circuit,[k_count[i] for i in range(nb_bits_position)]+[count[i] for i in range(nb_bits_position)],2*nb_bits_position,add_positions.control(),[continue_scanning,zeros[nb_bits_position-1]]+[loop[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left]+[zeros[i] for i in range(nb_bits_position-1)])
        #
        ##end if 1 == count[k_left]
        #end_if_equal(circuit,count,nb_bits_position,[one]+[zeros[i] for i in range(nb_bits_position-1)])
        
        #end if k_folding == k_count
        end_if_equal(circuit,k_count,nb_bits_position,k_folding)
        
        #if count[k_left] == 0
        #then there is no matching ) to find
        circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left])
    
    if truncation_point == 11 or truncation_point == "compute matching":
        return circuit,semantics
    
    #set k_count to its initial value by adding it 2**nb_bits_position - length
    first_one = -1
    for i,bit in enumerate(n_to_ctrl_state(2**nb_bits_position - length,nb_bits_position)):
        if '1'==bit:
            if 0 > first_one:
                first_one = i
            else:
                circuit.x(zeros[i])
    circuit.append(add_positions,[zeros[first_one]]+[k_count[i] for i in range(nb_bits_position)]+[(one if first_one == i else zeros[i]) for i in range(nb_bits_position)])
    
    if truncation_point == 12 or truncation_point == "restore k_count":
        return circuit,semantics
    
    #restore zeros
    first_one = -1
    for i,bit in enumerate(n_to_ctrl_state(2**nb_bits_position - length,nb_bits_position)):
        if '1'==bit:
            if 0 > first_one:
                first_one = i
            else:
                circuit.x(zeros[i])
    
    if truncation_point == 13 or truncation_point == "restore zeros":
        return circuit,semantics
    
    
    #Computed parameters
    #folding[kleft] -> continue_searching_right_matching_left[k_left] * k_right_matching_left[k_left] * loop                                 * k_count[k_left] * count[k_left]
    #(              -> 0 if a matching ) has been found               * k_right_matching_left         * number of . (at level 1) in the loop * k_left          * unspecified
    #(              -> 1 if a matching ) has not been found           * !=k_left                      * unspecified                          * k_left          * unspecified
    #. or )         -> 0                                              * k_left                        * 0                                    * k_left          * unspecified
    
    
    #4) Aggregating results
    
    #if each ( of the folding has a matching ) and if an extra ( at the beginning does not have a matching )
    #that is, if none of the conditions of incorrect_folding is fulfilled
    #then it is a correct_folding
    circuit.add_register(incorrect_folding)
    
    #checking if there is an exceeding )
    #there is if 0 == continue_searching_right_matching_left[k_left=0]
    #since the value 0==incorrect_folding is known then there is no need to add in order to increment
    #a CXGate() is sufficient
    circuit.append(MCXGate(nb_bits_position+1,ctrl_state="0"+n_to_ctrl_state(0,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left,incorrect_folding[0]])
    
    if truncation_point == 14 or truncation_point == "check incorrect )":
        return circuit,semantics
    
    #sum all the incorrect flags continue_searching_right_matching_left (1 per position) into incorrect_folding
    #the sum cannot overflow since if the flag continue_searching_right_matching_left[k_left=0] == 0 (at k_left=0 incorrect beeing 0) then there is a ) somewhere
    #so continue_searching_right_matching_left[k_left=somewhere] == 0
    #so incorrect_folding is not incremented at each position
    for k in range(1,length+1):
        circuit.append(add_positions.control(nb_bits_position,ctrl_state=n_to_ctrl_state(k,nb_bits_position)),[k_count[i] for i in range(nb_bits_position)]+[zeros[nb_bits_position-1],continue_searching_right_matching_left]+[zeros[i] for i in range(nb_bits_position-1)]+[incorrect_folding[i] for i in range(nb_bits_position)])
    
    if truncation_point == 15 or truncation_point == "check incorrect (":
        return circuit,semantics
    
    #it is a correct_folding if 0==incorrect_folding
    circuit.add_register(correct_folding)
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[incorrect_folding[i] for i in range(nb_bits_position)]+[correct_folding])
    
    return circuit,semantics

def matching_parenthesis_gate(length,truncation_point=-1):
    circ,_ = matching_parenthesis_circuit(length,truncation_point)
    return circ.to_gate(label="matching")

circ,_=matching_parenthesis_circuit(3)

circ.draw()

┌───┐┌───┐     ┌───┐                                        »
      (/./h/)_0: ┤ H ├┤ H ├─────┤ X ├──o───────────────────■─────────────────»
                 ├───┤└─┬─┘┌───┐└─┬─┘  │            ┌───┐  │                 »
      (/./h/)_1: ┤ H ├──┼──┤ H ├──┼────■────────────┤ X ├──o─────────────────»
                 ├───┤  │  └─┬─┘  │    │            └─┬─┘  │                 »
    k_folding_0: ┤ H ├──o────o────o────┼────■─────────┼────┼────■────────────»
                 ├───┤  │    │    │    │    │         │    │    │            »
    k_folding_1: ┤ H ├──o────o────o────┼────┼────■────┼────┼────┼────■───────»
                 └───┘               ┌─┴─┐  │    │    │    │    │    │       »
change_encoding: ────────────────────┤ X ├──┼────┼────■────┼────┼────┼───────»
                 ┌───┐               └───┘┌─┴─┐  │         │  ┌─┴─┐  │       »
      k_count_0: ┤ H ├────────────────────┤ X ├──┼─────────o──┤ X ├──┼────■──»
                 ├───┤                    └───┘┌─┴─┐       │  └───┘┌─┴─┐  │  »
      k_count_1: ┤ H ├─────────────────────────┤ X ├───────o───────┤ X ├──┼──»
                 └───┘                         └───┘     ┌─┴─┐     └───┘  │  »
        count_0: ────────────────────────────────────────┤ X ├──■─────────┼──»
                                                         └───┘  │         │  »
        count_1: ───────────────────────────────────────────────┼─────────┼──»
                                                                │       ┌─┴─┐»
      k_match_0: ───────────────────────────────────────────────┼───────┤ X ├»
                                                                │       └───┘»
      k_match_1: ───────────────────────────────────────────────┼────────────»
                 ┌───┐                                          │            »
            one: ┤ X ├──────────────────────────────────────────┼────────────»
                 └───┘                                          │            »
         zero_0: ───────────────────────────────────────────────┼────────────»
                                                                │            »
         zero_1: ───────────────────────────────────────────────┼────────────»
                                                              ┌─┴─┐          »
 continue_match: ─────────────────────────────────────────────┤ X ├──────────»
                 ┌───┐                                        └───┘          »
  continue_scan: ┤ X ├───────────────────────────────────────────────────────»
                 └───┘                                                       »
    incorrect_0: ────────────────────────────────────────────────────────────»
                                                                             »
    incorrect_1: ────────────────────────────────────────────────────────────»
                                                                             »
        correct: ────────────────────────────────────────────────────────────»
                                                                             »
«                                                       ┌────────────────┐     »
«      (/./h/)_0: ──────────────────────────────────────┤3               ├─────»
«                                                       │                │     »
«      (/./h/)_1: ──────────────────────────────────────┤4               ├─────»
«                                                       │                │     »
«    k_folding_0: ──────────────────────────────■───────┤                ├──■──»
«                                               │       │                │  │  »
«    k_folding_1: ──────────────────────────────┼────■──┤                ├──┼──»
«                                               │    │  │                │  │  »
«change_encoding: ──────────────────────────────┼────┼──┤                ├──┼──»
«                               ┌───────┐     ┌─┴─┐  │  │                │┌─┴─┐»
«      k_count_0: ──────────────┤3      ├──o─

statistics

In [7]:
print("width {} ; depth {}".format(circ.width(),circ.depth()))

width 19 ; depth 35


simulations

In [9]:
np.set_printoptions(threshold=np.inf)

all_truncation_points = ["any folding","any k_folding","candidate folding","encoding","init k_count","extend folding","init count","init k_right_matching_left","init cst","init continue_searching_right_matching_left","init continue_scanning","compute matching","restore k_count","restore zeros","check incorrect )","check incorrect ("]

evolution = []

farthest_truncation_points = 3
length = 3

for i,truncation_point in enumerate(all_truncation_points):
    if i <= farthest_truncation_points:
        backend = Aer.get_backend('statevector_simulator')
        circ,sem=matching_parenthesis_circuit(length,truncation_point)
        nb_bits = circ.width()
        job = execute(circ, backend=backend, shots=1, memory=True)
        job_result = job.result()
        state = np.asarray( [(sem(n_to_ctrl_state(i,nb_bits)),n_to_ctrl_state(i,nb_bits),('   ' if abs(val)<10.0**(-15) else '###')) for i,val in enumerate(np.asarray(job_result.get_statevector(circ)))])
        print(truncation_point)
        print(state)
        print("\n")

# Transpile for simulator
#simulator = Aer.get_backend('statevector_simulator')
#circ = transpile(circ, simulator)

# Run and get counts
#result = simulator.run(circ).result()
#result
#counts = result.get_counts(circ)
#plot_histogram(counts, title='Bell-State counts')

any folding
[['.' '00' '###']
 ['(' '01' '###']
 ['h' '10' '###']
 [')' '11' '###']]


any k_folding
[['0_kf  .' '0000' '###']
 ['0_kf  (' '0001' '###']
 ['0_kf  h' '0010' '###']
 ['0_kf  )' '0011' '###']
 ['1_kf  .' '0100' '###']
 ['1_kf  (' '0101' '###']
 ['1_kf  h' '0110' '###']
 ['1_kf  )' '0111' '###']
 ['2_kf  .' '1000' '###']
 ['2_kf  (' '1001' '###']
 ['2_kf  h' '1010' '###']
 ['2_kf  )' '1011' '###']
 ['3_kf  .' '1100' '###']
 ['3_kf  (' '1101' '###']
 ['3_kf  h' '1110' '###']
 ['3_kf  )' '1111' '###']]


candidate folding
[['0_kf  .' '0000' '   ']
 ['0_kf  (' '0001' '###']
 ['0_kf  h' '0010' '   ']
 ['0_kf  )' '0011' '   ']
 ['1_kf  .' '0100' '###']
 ['1_kf  (' '0101' '###']
 ['1_kf  h' '0110' '###']
 ['1_kf  )' '0111' '###']
 ['2_kf  .' '1000' '###']
 ['2_kf  (' '1001' '###']
 ['2_kf  h' '1010' '###']
 ['2_kf  )' '1011' '###']
 ['3_kf  .' '1100' '###']
 ['3_kf  (' '1101' '###']
 ['3_kf  h' '1110' '###']
 ['3_kf  )' '1111' '###']]


encoding
[['- 0_kf  .' '00000' '   ']
 ['- 